In [4]:
from devtools import debug
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)
!export PYTHONPATH=":./python"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# cSpell:disable

import re

from devtools import debug
from langchain.output_parsers import PydanticOutputParser
from langchain_community.document_loaders.web_base import WebBaseLoader
from pydantic import BaseModel, Field

URL = "https://www.agroparistech.fr/formations-ingenieur/mention-biodiversite-ecologie-evolution-montpellier-parcours-m1-biodiversite-vegetale-gestion-ecosystemes-tropicaux-montpellier"

loader = WebBaseLoader(URL)
doc = loader.load()
page = doc[0].page_content.strip()
page_cleaned = re.sub(r"[\t ]+", " ", page)
page_cleaned = re.sub(r"\n{3,}", "\n\n", page_cleaned)

In [6]:
from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt


class Parcours(BaseModel):
    metiers: list[str] = Field(
        description="liste de metiers auquels peut conduire la formation"
    )
    domaines: list[str] = Field(
        description="domaines auquels s'applique cette formation"
    )


system = """
    Extraire les metiers et les domaines d'une fiche de description de formation récupérée sur internet.  
    Repondre seulement avec les informations disponibles dans la description.
    \n
    """
user = """
    fiche de desctiption de formation:
    --- 
    {fiche}
    --- 
    {format_instructions}"""

parser = PydanticOutputParser(pydantic_object=Parcours)

prompt = def_prompt(system=system, user=user).partial(
    format_instructions=parser.get_format_instructions()
)

llm = get_llm(temperature=0.0)
chain = prompt | llm | parser
summaries = chain.batch([{"fiche": page_cleaned}], {"max_concurrency": 5})

debug(summaries)

2024-09-02 20:46:58.165 | INFO     | python.ai_core.llm:get_llm:319 - get LLM:'gpt_4omini_edenai' -configurable: False - streaming: False
2024-09-02 20:46:58.192 | INFO     | python.ai_core.cache:set_cache:54 - LLM cache : SQLiteCache


/tmp/ipykernel_168842/1225439580.py:36 <module>
    summaries: [
        Parcours(
            metiers=[
                'Chercheur en biodiversité',
                "Gestionnaire d'écosystèmes",
                'Consultant en environnement',
                'Enseignant-chercheur',
                'Expert en conservation',
                'Analyste en biologie végétale',
            ],
            domaines=[
                'Biodiversité',
                'Ecologie',
                'Gestion environnementale',
                'Sciences biologiques',
                'Sciences sociales',
                'Biogéographie',
            ],
        ),
    ] (list) len=1


[Parcours(metiers=['Chercheur en biodiversité', "Gestionnaire d'écosystèmes", 'Consultant en environnement', 'Enseignant-chercheur', 'Expert en conservation', 'Analyste en biologie végétale'], domaines=['Biodiversité', 'Ecologie', 'Gestion environnementale', 'Sciences biologiques', 'Sciences sociales', 'Biogéographie'])]